In [19]:
import requests,os,time
def ratelimit(dt):
    "A function that handles the rate of your calls."
    time.sleep(dt) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='selenium',session=False,path2selenium='/Users/lilly/Desktop/chromedriver',n_tries = 5,timeout=30,waiting_time=0.5):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    self.waiting_time = waiting_time # define simple rate_limit parameter.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Chrome(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, fName used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit(self.waiting_time)
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit(self.waiting_time)
      self.browser.get(url) # use selenium get method
      
      # Scroll down the page, in order to loade more 
      lenOfPage = self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
      match=False
      while(match==False):
          lastCount = lenOfPage
          time.sleep(1)
          lenOfPage = self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
          if lastCount==lenOfPage:
            match=True
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
      html=connector.browser.page_source
      return html,call_id

In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

logfile = 'logfile_test.txt' ## name your log file.
connector = Connector(logfile)

start_time = time.time()

url = 'https://www.vivino.com/explore?e=eJwNirsOgCAMAP-mM5o4dnPjE4wxtSJpImCgvv7eLnfDXarYQZKMDhK92A_OAX84emCDh9Ny3PGmKkHpgLJiJZUc28LlygoFt9AYHp1mW_lrZtEfxh4dMg=='

html, call_id =connector.get(url,"Test")

soup = BeautifulSoup(html, 'lxml')

print("--- %s seconds ---" % round((time.time() - start_time),2))

--- 21.06 seconds ---


In [99]:
# Finding the needed information in the soup and makes a dataframe
soup_select = soup.find_all('div', {'class': 'explorerCard__explorerCard--3Q7_0 explorerPageResults__explorerCard--3q6Qe'})
vivino='https://www.vivino.com'
urls=[]
names=[]
ratings=[]
region=[]

for i in soup_select:
    
    links = i.find_all('a', {'class': 'anchor__anchor--2QZvA'}) # find all a tags -connoting a hyperlink.
    urls.append([vivino+link['href'] for link in links if link.has_attr('href')][0])
    names.append(i.find_all('span', {'class': 'vintageTitle__wine--U7t9G'})[0].text)
    ratings.append(i.find_all('div', {'class': 'vivinoRatingWide__averageValue--1zL_5'})[0].text)
    region.append(i.find_all('a', {'class': 'anchor__anchor--2QZvA vintageLocation__anchor--T7J3k'})[2].text)

df=pd.DataFrame(zip(names,ratings,region,urls),columns=['name','rating','region','link'])
vivino=df.drop_duplicates().reset_index(drop=True)
vivino.to_csv('vivino_random.csv')

vivino

,name,rating,region,link
0,D.N.A 2014,4.7,Montefalco,https://www.vivino.com/alfredosa-dn/w/6618702?...
1,Tignanello 2016,4.6,Toscana,https://www.vivino.com/antinori-tuscany-tignan...
2,Tignanello 2017,4.6,Toscana,https://www.vivino.com/antinori-tuscany-tignan...
3,Valpolicella Superiore 2016,4.5,Valpolicella,https://www.vivino.com/ferragu-valpolicella-su...
4,Amarone 2009,4.5,Amarone della Valpolicella,https://www.vivino.com/le-guaite-di-noemi-le-g...
...,...,...,...,...
2020,Brunello di Montalcino Vigneto Manachiara 2012,4.2,Brunello di Montalcino,https://www.vivino.com/silvio-nardi-brunello-d...
2021,Giachini Barolo 2013,4.2,Barolo,https://www.vivino.com/giovanni-corino-giachin...
2022,Resa 56 Barolo 2010,4.2,Barolo,https://www.vivino.com/brandini-resa-56-barolo...
2023,Brunello di Montalcino 2014,4.2,Brunello di Montalcino,https://www.vivino.com/baricci-brunello-di-mon...


In [63]:
# Getting the 
df1=pd.read_csv('vivino.csv', index_col=[0])
df2=pd.read_csv('vivino_lowrating.csv', index_col=[0])

merged=pd.concat([df1,df2])
df3=merged.drop_duplicates().reset_index(drop=True)

df3.to_csv('vivino_v4.csv')


In [96]:
merged=pd.concat([df5,vivino])
df6=merge.drop_duplicates().reset_index(drop=True)
df6.to_csv('vivino_v7.csv')

In [102]:
df6

,name,rating,region,link
0,Raro Cabernet Sauvignon Selezione 2007,4.9,Veneto,https://www.vivino.com/marion-raro-cabernet-sa...
1,Barolo Riserva Monfortino 1985,4.9,Barolo,https://www.vivino.com/giacomo-conterno-barolo...
2,La Fabriseria Amarone della Valpolicella Class...,4.9,Amarone della Valpolicella Classico,https://www.vivino.com/agricola-flli-tedeschi-...
3,Barolo Lazzarito 2000,4.9,Barolo,https://www.vivino.com/vietti-barolo-lazzarito...
4,Basaltico Amarone della Valpolicella 2004,4.9,Amarone della Valpolicella,https://www.vivino.com/ernesto-ruffo-basaltico...
...,...,...,...,...
8570,Brunello di Montalcino Vigneto Manachiara 2012,4.2,Brunello di Montalcino,https://www.vivino.com/silvio-nardi-brunello-d...
8571,Giachini Barolo 2013,4.2,Barolo,https://www.vivino.com/giovanni-corino-giachin...
8572,Resa 56 Barolo 2010,4.2,Barolo,https://www.vivino.com/brandini-resa-56-barolo...
8573,Brunello di Montalcino 2014,4.2,Brunello di Montalcino,https://www.vivino.com/baricci-brunello-di-mon...
